In [1]:
import finrid_api_get as fapi
import db_functions as dbf
import pandas as pd
import numpy as np

In [2]:
CONN = dbf.create_connection()

3.32.3


In [3]:
def get_df_db(conn, table, sql = None):
    curs = conn.cursor()
    sql = sql if sql else f'SELECT * FROM {table}'
    res = curs.execute(sql).fetchall()
    return res

def delete_full_table(table_name):
    CONN.cursor().execute(f'DELETE FROM {str(table_name)}')
    CONN.commit()

## Take data from API to db


In [4]:
wind_data = fapi.get_data_from_api(75)
wind_df = pd.json_normalize(wind_data)

consumption_data = fapi.get_data_from_api(124)
consumption_df = pd.json_normalize(consumption_data)
consumption_df = consumption_df[consumption_df['start_time'].isin(wind_df['start_time'])].reset_index(drop = True)

#wind_df.to_sql('wind_power_generation', con=CONN, if_exists='append', index = False)
#consumption_df.to_sql('electricity_consumption', con=CONN, if_exists='append', index = False)

In [5]:
def time_helper(df_date_col, time_value):
    if time_value == 'hour':
        return pd.to_datetime(df_date_col).dt.hour
    if time_value == 'day':
        return pd.to_datetime(df_date_col).dt.day
    if time_value == 'month':
        return pd.to_datetime(df_date_col).dt.month
    if time_value == 'year':
        return pd.to_datetime(df_date_col).dt.year
        

df_hourly = pd.DataFrame(columns=['percentage'])
df_hourly['percentage'] = np.array(wind_df['value']/consumption_df['value'])
time_list = ['hour', 'day', 'month', 'year']
for i in time_list:
    df_hourly[i] = time_helper(wind_df['start_time'], i)

df_hourly


,percentage,hour,day,month,year
0,0.140563,0,1,1,2019
1,0.136290,1,1,1,2019
2,0.119553,2,1,1,2019
3,0.100979,3,1,1,2019
4,0.082378,4,1,1,2019
...,...,...,...,...,...
8746,0.130384,20,31,12,2019
8747,0.130022,20,31,12,2019
8748,0.135308,22,31,12,2019
8749,0.127671,23,31,12,2019


In [6]:
df_avg = wind_df.copy()
df_avg['value'] = wind_df['value']/consumption_df['value']
df_avg['start_time'] = pd.to_datetime(df_avg['start_time'])
df_avg = df_avg.drop('end_time', axis = 1)
df_avg_day = df_avg.resample('D', on = 'start_time').mean()
df_avg_mon= df_avg.resample('M', on = 'start_time').mean()
df_avg_day = df_avg_day.reset_index()
df_avg_mon = df_avg_mon.reset_index()

#df_avg.set_index('start_time').groupby(pd.Grouper(freq='D')).mean()

In [7]:
for i in ['month', 'year']:
    df_avg_mon[i] = time_helper(df_avg_mon['start_time'], i)
df_avg_mon = df_avg_mon.drop('start_time', axis= 1)
df_avg_mon

,value,month,year
0,0.049300,1,2019
1,0.085143,2,2019
2,0.084968,3,2019
3,0.068600,4,2019
4,0.071623,5,2019
5,0.077015,6,2019
6,0.048990,7,2019
7,0.072341,8,2019
8,0.068627,9,2019
9,0.063989,10,2019


In [12]:
df_avg_mon
df_avg_mon.columns = ['percentage','month','year']
df_avg_mon

,percentage,month,year
0,0.049300,1,2019
1,0.085143,2,2019
2,0.084968,3,2019
3,0.068600,4,2019
4,0.071623,5,2019
5,0.077015,6,2019
6,0.048990,7,2019
7,0.072341,8,2019
8,0.068627,9,2019
9,0.063989,10,2019


In [9]:
for i in ['day','month', 'year']:
    df_avg_day[i] = time_helper(df_avg_day['start_time'], i)
df_avg_day = df_avg_day.drop('start_time', axis= 1)
df_avg_day

,value,day,month,year
0,0.087050,1,1,2019
1,0.110791,2,1,2019
2,0.072691,3,1,2019
3,0.026313,4,1,2019
4,0.009846,5,1,2019
...,...,...,...,...
360,0.077387,27,12,2019
361,0.117907,28,12,2019
362,0.165220,29,12,2019
363,0.153762,30,12,2019


In [13]:
df_avg_day.columns = ['percentage','day','month','year']
df_avg_day

,percentage,day,month,year
0,0.087050,1,1,2019
1,0.110791,2,1,2019
2,0.072691,3,1,2019
3,0.026313,4,1,2019
4,0.009846,5,1,2019
...,...,...,...,...
360,0.077387,27,12,2019
361,0.117907,28,12,2019
362,0.165220,29,12,2019
363,0.153762,30,12,2019


In [14]:
df_avg_day.to_sql('daily_wind_coverage', con=CONN, if_exists='append', index = False)
df_avg_mon.to_sql('monthly_wind_coverage', con=CONN, if_exists='append', index = False)
df_hourly.to_sql('hourly_wind_coverage', con=CONN, if_exists='append', index = False)